# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 200  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 468
Trainable params: 449
Non-trainable params: 19
______________________________________________________________

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/200
3122/3122 - 5s - loss: 0.6786 - val_loss: 0.3813 - 5s/epoch - 2ms/step
Epoch 2/200
3122/3122 - 5s - loss: 0.3206 - val_loss: 0.2828 - 5s/epoch - 1ms/step
Epoch 3/200
3122/3122 - 3s - loss: 0.2538 - val_loss: 0.2357 - 3s/epoch - 959us/step
Epoch 4/200
3122/3122 - 5s - loss: 0.2177 - val_loss: 0.2073 - 5s/epoch - 2ms/step
Epoch 5/200
3122/3122 - 5s - loss: 0.1937 - val_loss: 0.1862 - 5s/epoch - 2ms/step
Epoch 6/200
3122/3122 - 5s - loss: 0.1762 - val_loss: 0.1709 - 5s/epoch - 1ms/step
Epoch 7/200
3122/3122 - 5s - loss: 0.1632 - val_loss: 0.1592 - 5s/epoch - 1ms/step
Epoch 8/200
3122/3122 - 5s - loss: 0.1525 - val_loss: 0.1486 - 5s/epoch - 2ms/step
Epoch 9/200
3122/3122 - 4s - loss: 0.1439 - val_loss: 0.1409 - 4s/epoch - 1ms/step
Epoch 10/200
3122/3122 - 4s - loss: 0.1365 - val_loss: 0.1341 - 4s/epoch - 1ms/step
Epoch 11/200
3122/3122 - 4s - loss: 0.1306 - val_loss: 0.1284 - 4s/epoch - 1ms/step
Epoch 12/200
3122/3122 - 4s - loss: 0.1259 - val_loss: 0.1251 - 4s/epoch - 1ms/step

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [6]:
fit = np.load("data/fit.npz")

In [7]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)